13 ЛАБ. РАБОТА

In [80]:
import pandas as pd

import spacy
import ru_core_news_lg
nlp = ru_core_news_lg.load()

Данные

In [81]:
import os
import chardet

folder_path = "./Pushkin/"
data = []

# Функция для определения кодировки текстового файла
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Читаем каждый файл в папке и добавляем его содержимое в список
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        encoding = detect_encoding(file_path)
        
        with open(file_path, 'r', encoding=encoding) as file:
            poem_text = file.read()
            poem_id = os.path.splitext(filename)[0]  # Используем имя файла (без расширения) как id
            data.append((poem_id, poem_text))

# Создаем DataFrame из списка данных
columns = ['id', 'text']
df = pd.DataFrame(data, columns=columns)
df['id'] = df.index
df['text'] = df['text'].str.replace('Александр Пушкин\n', '')
# Выводим первые несколько строк DataFrame
df.head(10)


,id,text
0,0,"19 ОКТЯБРЯ\nРоняет лес багряный свой убор,\nСр..."
1,1,\n Александр Сергеевич Пушкин\n РУСЛ...
2,2,СКУПОЙ РЫЦАРЬ\n(Сцены из Ченстоновой трагикоме...
3,3,СКАЗКА\nО РЫБАКЕ И РЫБКЕ\nЖил старик со своею ...
4,4,"СКАЗКА О ЦАРЕ САЛТАНЕ,\nО СЫНЕ ЕГО СЛАВНОМ И М..."
5,5,СКАЗКА\nО ПОПЕ И О РАБОТНИКЕ ЕГО БАЛДЕ\nЖил-бы...
6,6,"СКАЗКА\nО ЗОЛОТОМ ПЕТУШКЕ\nНегде, в тридевятом..."
7,7,СКАЗКА\nО МЕРТВОЙ ЦАРЕВНЕ\nИ О СЕМИ БОГАТЫРЯХ\...
8,8,СКАЗКА О МЕДВЕДИХЕ\nКак весенней теплою порою\...
9,9,УЗНИК\nСижу за решеткой в темнице сырой.\nВско...


In [82]:
folder_path = "./Nekrasov/"
data = []

# Функция для определения кодировки текстового файла
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Читаем каждый файл в папке и добавляем его содержимое в список
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)
        encoding = detect_encoding(file_path)
        
        with open(file_path, 'r', encoding=encoding) as file:
            poem_text = file.read()
            poem_id = os.path.splitext(filename)[0]  # Используем имя файла (без расширения) как id
            data.append((poem_id, poem_text))

# Создаем DataFrame из списка данных
columns = ['id', 'text']
df2 = pd.DataFrame(data, columns=columns)
df2['id'] = df2.index
df2['text'] = df2['text'].str.replace('Николай Некрасов\n', '')
# Выводим первые несколько строк DataFrame
df2.head(10)

,id,text
0,0,РУССКИЕ ЖЕНЩИНЫ\nКНЯГИНЯ ТРУБЕЦКАЯ\nПОЭМА\n(18...
1,1,РАЗМЫШЛЕНИЯ У ПАРАДНОГО ПОДЪЕЗДА\nВот парадный...
2,2,ТРОЙКА\nЧто ты жадно глядишь на дорогу\nВ стор...
3,3,ЭЛЕГИЯ\nА. Н. Е⟨рако⟩ву\nПускай нам говорит из...
4,4,"* * *\nЯ не люблю иронии твоей,\nОставь ее отж..."
5,5,"В ДОРОГЕ\n«Скучно! скучно!.. Ямщик удалой,\nРа..."
6,6,"* * *\nВчерашний день, часу в шестом,\n ..."
7,7,"ДЕДУШКА\nПосвящается З-н-ч-е\nI\nРаз у отца, в..."
8,8,ЖЕЛЕЗНАЯ ДОРОГА\nВаня (в кучерском армячке). П...
9,9,КРЕСТЬЯНСКИЕ ДЕТИ\nОпять я в деревне. Хожу на ...


Для Пушкина

In [83]:
#список из частей речи в тексте
df['new_col'] = df['text'].apply(lambda x:nlp(x))
df['new_col'] = df['new_col'].apply(lambda x: [token.tag_ for token in x])

#колонки части речи
tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'ENTITY', 'NOUN', 'PART', 'PRON', 'PUNCT', 'SCONJ', 'VERB']
for i in tags:
    df[i] = df['new_col'].apply(lambda x: x.count(i)) #заполняем кол-вом вхождений

#Доля частей речи в тексте
df['summ'] = df.iloc[:,-13:-1].sum(axis=1) #суммируем значения 
df['summ'] = df['summ'].replace(0, 1)
for i in tags:
    df[i] = df[i]/df['summ'] #приведение значений к доле или процентному соотношению относительно общей суммы

#Нормализация
for column in tags:
    df[column] = (df[column] -df[column].mean()) / (df[column].abs().max()) #центрирование и шкалирование
df['class'] = 1
df.drop(['summ', 'text','new_col','id','ENTITY'], axis=1, inplace=True)
df.head()

,ADJ,ADP,ADV,AUX,CCONJ,DET,NOUN,PART,PRON,PUNCT,SCONJ,VERB,class
0,0.046650,0.083072,-0.073069,0.026143,0.000785,0.206253,0.002934,-0.098034,0.093322,-0.080184,-0.146291,-0.034302,1
1,0.093155,0.051650,0.024069,-0.093188,-0.017899,-0.049995,0.018150,-0.112623,-0.108743,0.047054,-0.103210,0.040467,1
2,-0.223890,-0.108729,0.036741,0.123013,-0.080822,0.152901,-0.144856,0.119337,0.289695,0.061525,0.136393,0.052101,1
3,-0.193943,0.036922,-0.085318,0.232549,-0.257231,-0.264461,0.026643,0.119603,0.096473,0.157878,-0.133820,0.253139,1
4,-0.212316,0.175133,-0.029494,-0.095630,-0.003467,-0.055895,0.039166,0.083708,-0.030222,0.041031,-0.136022,0.139299,1


Для Некрасова

In [84]:
#список из частей речи в тексте
df2['new_col'] = df2['text'].apply(lambda x:nlp(x))
df2['new_col'] = df2['new_col'].apply(lambda x: [token.tag_ for token in x])

#колонки части речи
tags = ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'ENTITY', 'NOUN', 'PART', 'PRON', 'PUNCT', 'SCONJ', 'VERB']
for i in tags:
    df2[i] = df2['new_col'].apply(lambda x: x.count(i)) #заполняем кол-вом вхождений

#Доля частей речи в тексте
df2['summ'] = df2.iloc[:,-13:-1].sum(axis=1) #суммируем значения 
df2['summ'] = df2['summ'].replace(0, 1)
for i in tags:
    df2[i] = df2[i]/df2['summ'] #приведение значений к доле или процентному соотношению относительно общей суммы

#Нормализация
for column in tags:
    df2[column] = (df2[column] -df2[column].mean()) / (df2[column].abs().max()) #центрирование и шкалирование
df2['class'] = 0
df2.drop(['summ', 'text','new_col','id','ENTITY'], axis=1, inplace=True)
df2.head()

,ADJ,ADP,ADV,AUX,CCONJ,DET,NOUN,PART,PRON,PUNCT,SCONJ,VERB,class
0,-0.153227,-0.065201,0.083718,0.303804,-0.054969,0.066098,-0.053747,-0.043159,0.281772,0.006755,-0.045956,0.085243,0
1,0.098898,0.101105,-0.137246,-0.150176,-0.053808,0.214385,0.114680,-0.131391,0.064307,-0.084970,-0.054533,-0.091161,0
2,0.226898,0.273672,-0.097100,-0.188999,0.263440,0.035788,0.133936,-0.019166,-0.178772,-0.214244,-0.094271,0.137282,0
3,0.062539,-0.008590,-0.213847,0.317538,0.028221,-0.103635,0.159964,0.026033,0.019652,-0.083353,0.038169,0.138139,0
4,0.259282,-0.253367,0.590766,-0.473236,0.490908,-0.371794,-0.209660,-0.186404,0.374876,-0.155542,-0.216159,-0.146644,0


Объединяем датафреймы

In [85]:
df_general = pd.concat([df,df2], sort=False)
df_general

,ADJ,ADP,ADV,AUX,CCONJ,DET,NOUN,PART,PRON,PUNCT,SCONJ,VERB,class
0,0.046650,0.083072,-0.073069,0.026143,0.000785,0.206253,0.002934,-0.098034,0.093322,-0.080184,-0.146291,-0.034302,1
1,0.093155,0.051650,0.024069,-0.093188,-0.017899,-0.049995,0.018150,-0.112623,-0.108743,0.047054,-0.103210,0.040467,1
2,-0.223890,-0.108729,0.036741,0.123013,-0.080822,0.152901,-0.144856,0.119337,0.289695,0.061525,0.136393,0.052101,1
3,-0.193943,0.036922,-0.085318,0.232549,-0.257231,-0.264461,0.026643,0.119603,0.096473,0.157878,-0.133820,0.253139,1
4,-0.212316,0.175133,-0.029494,-0.095630,-0.003467,-0.055895,0.039166,0.083708,-0.030222,0.041031,-0.136022,0.139299,1
5,-0.293850,0.004480,0.076662,0.043067,-0.171318,0.039068,-0.061981,0.221672,0.017438,0.202720,-0.152528,0.248595,1
6,-0.220349,0.078956,0.043205,0.040569,-0.029388,0.017816,-0.017475,-0.063979,-0.016134,0.136972,-0.104536,0.110044,1
7,-0.153274,0.108537,-0.017092,0.017698,-0.012086,-0.040967,-0.123302,0.092604,0.102368,0.121965,-0.153751,0.153971,1
8,-0.082667,0.117549,-0.110819,-0.152068,-0.239725,-0.180617,0.128407,0.105549,0.054442,-0.014987,-0.025642,-0.004741,1
9,-0.094080,0.022792,0.186664,-0.224958,-0.034424,-0.109496,0.000992,-0.001686,-0.181844,0.149120,-0.070597,0.016359,1


Деление на тестовую и общучающую выборки

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_general.drop(['class'], axis=1), df_general['class'], test_size=0.2, random_state=42)

SVM (метод опорных векторов)

In [87]:
from sklearn import svm

classifier = svm.SVC() #создание
classifier.fit(X_train, y_train) #обучение
y_predict = classifier.predict(X_test)

KnN (k ближайших соседей)

In [92]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, y_train)
y_predict2 = knn_model.predict(X_test)

RandomForest (случайные леса)    

In [113]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=50, random_state=42) #n_estimators - количество деревьев в случайном лесу
random_forest_model.fit(X_train, y_train.ravel()) # ravel - преобразования многомерного массива в одномерный
y_predict4 = random_forest_model.predict(X_test)


GaussianNB (Наивный байесовский классификатор )

In [96]:
from sklearn.naive_bayes import GaussianNB

naive_bayes_model = GaussianNB()
naive_bayes_model.fit(X_train, y_train)
y_predict5 = naive_bayes_model.predict(X_test)

Рассчёт метрик

In [115]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

print(f"Precision SVM: {precision_score(y_test,y_predict, average='weighted')}")
print(f"Precision kNN: {precision_score(y_test,y_predict2, average='weighted')}")
print(f"Precision Random Forest: {precision_score(y_test,y_predict4, average='weighted')}")
print(f"Precision Gaussian NB: {precision_score(y_test,y_predict5, average='weighted')} \n")

print(f"Recall SVM: {recall_score(y_test,y_predict, average='weighted')}")
print(f"Recall kNN: {recall_score(y_test,y_predict2, average='weighted')}")
print(f"Recall Random Forest: {recall_score(y_test,y_predict4, average='weighted')}")
print(f"Recall Gaussian NB: {recall_score(y_test,y_predict5, average='weighted')} \n")

print(f"f1 SVM: {f1_score(y_test,y_predict, average='weighted')}")
print(f"f1 kNN: {f1_score(y_test,y_predict2, average='weighted')}")
print(f"f1 Random Forest: {f1_score(y_test,y_predict4, average='weighted')}")
print(f"f1 Gaussian NB: {f1_score(y_test,y_predict5, average='weighted')} \n")

print(f"Roc AUC SVM: {roc_auc_score(y_test,y_predict, average='weighted')}")
print(f"Roc AUC kNN: {roc_auc_score(y_test,y_predict2, average='weighted')}")
print(f"Roc AUC Random Forest: {roc_auc_score(y_test,y_predict4, average='weighted')}")
print(f"Roc AUC Gaussian NB: {roc_auc_score(y_test,y_predict5, average='weighted')}")

Precision SVM: 0.48999999999999994
Precision kNN: 0.675
Precision Random Forest: 0.675
Precision Gaussian NB: 0.4375 

Recall SVM: 0.7
Recall kNN: 0.7
Recall Random Forest: 0.7
Recall Gaussian NB: 0.5 

f1 SVM: 0.5764705882352941
f1 kNN: 0.6799999999999999
f1 Random Forest: 0.6799999999999999
f1 Gaussian NB: 0.4666666666666666 

Roc AUC SVM: 0.5
Roc AUC kNN: 0.5952380952380953
Roc AUC Random Forest: 0.5952380952380953
Roc AUC Gaussian NB: 0.35714285714285715


C:\Users\79526\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
